# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list
- Assigning trophies
- Assigning fines
- Calculating total roster value to determine the salary, extra budget and luxury tax
- Update team_real
- TBD
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC:
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm
    

In [6]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import progressbar
with open('credential.json','r') as f:
    cred = json.load(f)
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,...,Au,Qt. A,Qt. I,Diff.,Classe,Eta',Nome Completo,Nazionalit'a,Nome Squadra,Allenatore
2610,2610,A,RONALDO,Juventus,10,6.50,9.80,8,0,0,...,0,49,48,1,05/02/1985,35,Cristiano Ronaldo,Portogallo,AS 800A,enzo
2531,2531,A,LUKAKU,Inter,13,6.71,9.46,8,0,0,...,0,46,42,4,13/03/1993,27,Lukaku Romelu,"Belgio, Repubblica Democratica del Congo",SOROS FC,musci8
785,785,A,IMMOBILE,Lazio,12,6.46,8.46,7,0,0,...,0,40,49,-9,20/02/1990,30,Immobile Ciro,Italia,PALLA PAZZA,nanni
441,441,A,BELOTTI,Torino,13,6.85,9.04,7,0,0,...,0,37,29,8,20/12/1993,27,Belotti Andrea,Italia,XYZ,luca
2530,2530,A,IBRAHIMOVIC,Milan,6,7.25,11.25,8,0,0,...,0,35,35,0,03/10/1981,39,Ibrahimovic Zlatan,Svezia,XYZ,luca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,2013,A,RICCI F.,Sassuolo,1,0.00,0.00,0,0,0,...,0,1,1,0,27/05/1994,26,Ricci Federico,Italia,svincolato,None
4387,4387,A,ADORANTE,Parma,1,0.00,0.00,0,0,0,...,0,1,1,0,05/02/2000,20,Adorante Andrea,Italia,svincolato,None
5322,5322,A,MIHAILA,Parma,0,0.00,0.00,0,0,0,...,0,1,3,-2,02/02/2000,20,Mihaila Valentin,Romania,svincolato,None
4335,4335,A,SCHIAPPACASSE,Sassuolo,1,6.00,6.00,0,0,0,...,0,1,1,0,12/01/1999,21,Schiappacasse Nicolas,"Uruguay, Italia",SOROS FC - PRIMAVERA,musci8


In [3]:
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


## Update personal info
this part is to run through the players in our database and cross check the info with the stats db:
- if the player is not in our db, we don't care: we'll add it when someone wants to buy them
- if the player is in our db but not in stats it means that it left serie A, so the algorithm should change the team to none:
```collection.update_one({'_id': pl['_id']},{'$set':{'info.personal_info.team_real': None}})
```
- if the player has changed teams we need to update the information
- update name if it has changed, based on id

In [30]:
def update_personal_info(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]

    player_out = []
    posts = collection.find({})
    stats.index = list(stats['Id'])
    for dic in posts:
        Id = dic['_id']
        if Id in stats.index:
            collection.update_one({'_id': Id},{'$set':{'name': stats.loc[Id].Nome }})
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': stats.loc[Id].Squadra}})
        else:
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': None}})
            player_out.append(collection.find_one({'_id': Id})['name'])
    
    return player_out

In [31]:
update_personal_info(stats)

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


['EMPEREUR']

## UPDATE stats

In [9]:
def update_stats(stats, Id, CR = cred['cred'], DB = 'Game', CO = 'Players'):

    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    stats.index = list(stats.Id)
    #print(stats[stats['Id'] == int(Id)].T[0][4:20])
    stats_dict = dict(stats[stats['Id'] == int(Id)].T[Id][4:20])

    collection.update_one({'_id': Id}, {'$set': {'info.stats': stats_dict}})

In [10]:
def update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    all_pl = list(collection.find({}))
    
    for pl in progressbar.progressbar(all_pl):
        #print(pl['_id'])
        if len(stats[stats['Id'] == pl['_id']]) == 0:
            print(pl['name']+' left serie A')
            #
            continue
        else:
            update_stats(stats, pl['_id'], CR, DB, CO)

In [5]:
# algorithm runs through the entire db and updates stats value
# if player no more in serie a it prints a message, but we need to include change of team_real to none
#it is probably best to 
update_stats_all(stats)

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))
 65% (368 of 563) |##############        | Elapsed Time: 0:01:19 ETA:   0:02:07

EMPEREURleft serie a, modify algorithm to change team_real to None. profile not in FC db anymore


100% (563 of 563) |######################| Elapsed Time: 0:01:58 Time:  0:01:58


## Add player to database

## Manage roster value and extra budget/luxury tax